## Inference
### Анализ последствий наводнений
#### Команда DzerAI

In [1]:
import rasterio, os
import torch
import numpy as np
from tqdm import tqdm


In [2]:

with rasterio.open('train/images/5.tif') as multi_band_src:
    image = multi_band_src.read()
    meta = multi_band_src.meta

meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 3230,
 'height': 2336,
 'count': 10,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.00015075417956656787, 0.0, 142.827904,
        0.0, -8.649101027397402e-05, 52.83709)}

In [3]:
import torch
import numpy as np
from tqdm import tqdm

with rasterio.open('train/images/5.tif') as multi_band_src:
    image = multi_band_src.read()
    meta = multi_band_src.meta

meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 3230,
 'height': 2336,
 'count': 10,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.00015075417956656787, 0.0, 142.827904,
        0.0, -8.649101027397402e-05, 52.83709)}

In [5]:
import torch
import numpy as np
from tqdm import tqdm

with rasterio.open('train/images/5.tif') as multi_band_src:
    image = multi_band_src.read()
    meta = multi_band_src.meta

meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 3230,
 'height': 2336,
 'count': 10,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.00015075417956656787, 0.0, 142.827904,
        0.0, -8.649101027397402e-05, 52.83709)}

In [6]:

with rasterio.open('train/images/5.tif') as multi_band_src:
    image = multi_band_src.read()
    meta = multi_band_src.meta

meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 3230,
 'height': 2336,
 'count': 10,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.00015075417956656787, 0.0, 142.827904,
        0.0, -8.649101027397402e-05, 52.83709)}

In [7]:
predict = np.zeros((meta['height'], meta['width']), dtype=np.uint16)

model = torch.load("best_model_0.pt").to('cuda')
model.eval();

In [8]:

with torch.no_grad():
    for y in tqdm(range(0, meta['height'], 256)):
        for x in range(0, meta['width'], 256):
            tile = image[:, y:y+256, x:x+256]
            if x+256 > meta['width']:
                padx = -meta['width']%256
                tile = np.pad(tile, ((0,0), (0,0), (0, padx)))
            if y+256 > meta['height']:
                pady = -meta['height']%256
                tile = np.pad(tile, ((0,0), (0, pady), (0, 0)))
            tile = torch.from_numpy(tile).float()
            pred = model(tile.unsqueeze(0).to('cuda'))
            pred = pred[0,0].to('cpu').numpy()
            pred = pred.astype(np.uint16)
            if x+256 > meta['width']:
                pred = pred[:, :-padx]
            if y+256 > meta['height']:
                pred = pred[:-pady, :]
            predict[y:y+256, x:x+256] = pred


100%|██████████| 10/10 [00:08<00:00,  1.16it/s]


In [9]:
os.makedirs('out', exist_ok=True)

meta['count'] = 1

with rasterio.open('out/5.tif', 'w', **meta) as fout:
    fout.write(predict, 1)

In [10]:
# Отправка решения для тестового набора:

In [11]:
os.makedirs('out/images', exist_ok=True)
for name in os.listdir('test_scoltech/images/'):
    with rasterio.open(f'test_scoltech/images/{name}') as multi_band_src:
        image = multi_band_src.read()
        meta = multi_band_src.meta
    predict = np.zeros((meta['height'], meta['width']), dtype=np.uint16)
    with torch.no_grad():
        for y in tqdm(range(0, meta['height'], 256)):
            for x in range(0, meta['width'], 256):
                tile = image[:, y:y+256, x:x+256]
                if x+256 > meta['width']:
                    padx = -meta['width']%256
                    tile = np.pad(tile, ((0,0), (0,0), (0, padx)))
                if y+256 > meta['height']:
                    pady = -meta['height']%256
                    tile = np.pad(tile, ((0,0), (0, pady), (0, 0)))
                tile = torch.from_numpy(tile).float()
                pred = model(tile.unsqueeze(0).to('cuda'))
                pred = pred[0,0].to('cpu').numpy()
                pred = pred.astype(np.uint16)
                if x+256 > meta['width']:
                    pred = pred[:, :-padx]
                if y+256 > meta['height']:
                    pred = pred[:-pady, :]
                predict[y:y+256, x:x+256] = pred
    meta['count'] = 1
    with rasterio.open(f'out/images/{name}', 'w', **meta) as fout:
        fout.write(predict, 1)

100%|██████████| 20/20 [00:14<00:00,  1.38it/s]
